# Import Ports
This example shows how to import ports. In this example, we are going to

- Download an example board
- Create a configuration file
  - Add a circuit port between two nets
  - Add a circuit port between two pins
  - Add a circuit port between two pin groups
  - Add a circuit port between two coordinates
  - Add a coax port
  - Add a port reference to the nearest pin
  - Add distributed ports
- Import the configuration file

## Perform imports and define constants

Perform required imports.

In [1]:
import json
import toml
from pathlib import Path
import tempfile

from ansys.aedt.core.examples.downloads import download_file
from pyedb import Edb

Define constants.

In [2]:
AEDT_VERSION = "2025.2"
NG_MODE = False

Download the example PCB data.

In [3]:
temp_folder = tempfile.TemporaryDirectory(suffix=".ansys")
file_edb = download_file(source="edb/ANSYS-HSD_V1.aedb", local_path=temp_folder.name)

## Load example layout

In [4]:
edbapp = Edb(file_edb, edbversion=AEDT_VERSION)

C:\actions-runner\_work\pyaedt-examples\pyaedt-examples\.venv\lib\site-packages\pyedb\misc\decorators.py:55: UserWarning: Argument `edbversion` is deprecated for method `Edb`; use `version` instead.
  warnings.warn(
C:\actions-runner\_work\pyaedt-examples\pyaedt-examples\.venv\lib\site-packages\pyedb\generic\design_types.py:301: UserWarning: Your ANSYS AEDT version is eligible to gRPC version.You might consider switching to that version for better user experience.For more information please check this link: https://edb.docs.pyansys.com/version/dev/grpc_api/index.html
  warnings.warn(GRPC_GENERAL_WARNING, UserWarning)


PyEDB INFO: Star initializing Edb 10:29:25.787621


PyEDB INFO: Edb version 2025.2


PyEDB INFO: Logger is initialized. Log file is saved to C:\Users\ansys\AppData\Local\Temp\pyedb_ansys.log.


PyEDB INFO: legacy v0.67.1


...

PyEDB INFO: Database ANSYS-HSD_V1.aedb Opened in 2025.2


PyEDB INFO: Cell main Opened


PyEDB INFO: Builder was initialized.


PyEDB INFO: open_edb completed in 9.0701 seconds.


PyEDB INFO: EDB initialization completed in 9.1499 seconds.


## Create an empty dictionary to host all configurations

In [5]:
cfg = dict()

## Add a circuit port between two nets

Keywords

- **name**. Name of the port.
- **Reference_designator**. Reference designator of the component.
- **type**. Type of the port. Supported types are 'circuit', 'coax', 'gap_port', 'wave_port', 'diff_wave_port'.
- **impedance**. Impedance of the port. Default is 50 Ohm.
- **positive_terminal**. Positive terminal of the port. Supported types are 'net', 'pin', 'pin_group', 'coordinates'.
- **negative_terminal**. Negative terminal of the port. Supported types are 'net', 'pin', 'pin_group', 'coordinates',
'nearest_pin'.

In [6]:
port_1 = {
    "name": "port_1",
    "reference_designator": "X1",
    "type": "circuit",
    "positive_terminal": {"net": "PCIe_Gen4_TX2_N"},
    "negative_terminal": {"net": "GND"},
}

## Add a circuit port between two pins

In [7]:
port_2 = {
    "name": "port_2",
    "reference_designator": "C375",
    "type": "circuit",
    "positive_terminal": {"pin": "1"},
    "negative_terminal": {"pin": "2"},
}

## Add a circuit port between two pin groups

In [8]:
pin_groups = [
    {"name": "U9_5V_1", "reference_designator": "U9", "pins": ["32", "33"]},
    {"name": "U9_GND", "reference_designator": "U9", "net": "GND"},
]

In [9]:
port_3 = {
    "name": "port_3",
    "type": "circuit",
    "positive_terminal": {"pin_group": "U9_5V_1"},
    "negative_terminal": {"pin_group": "U9_GND"},
}

## Add a circuit port between two coordinates

Keywords

- **layer**. Layer on which the terminal is placed
- **point**. XY coordinate the terminal is placed
- **net**. Name of the net the terminal is placed on

In [10]:
port_4 = {
    "name": "port_4",
    "type": "circuit",
    "positive_terminal": {"coordinates": {"layer": "1_Top", "point": ["104mm", "37mm"], "net": "AVCC_1V3"}},
    "negative_terminal": {"coordinates": {"layer": "Inner6(GND2)", "point": ["104mm", "37mm"], "net": "GND"}},
}

## Add a coax port

In [11]:
port_5 = {"name": "port_5", "reference_designator": "U1", "type": "coax", "positive_terminal": {"pin": "AM17"}}

## Add a port reference to the nearest pin

Keywords

- **reference_net**. Name of the reference net
- **search_radius**. Reference pin search radius in meter

In [12]:
port_6 = {
    "name": "port_6",
    "reference_designator": "U15",
    "type": "circuit",
    "positive_terminal": {"pin": "D7"},
    "negative_terminal": {"nearest_pin": {"reference_net": "GND", "search_radius": 5e-3}},
}

## Add distributed ports

Keywords

- **distributed**. Whether to create distributed ports. When set to True, ports are created per pin

In [13]:
ports_distributed = {
    "name": "ports_d",
    "reference_designator": "U7",
    "type": "circuit",
    "distributed": True,
    "positive_terminal": {"net": "VDD_DDR"},
    "negative_terminal": {"net": "GND"},
}

## Add setups in configuration

In [14]:
cfg["pin_groups"] = pin_groups
cfg["ports"] = [port_1, port_2, port_3, port_4, port_5, port_6, ports_distributed]

## Write configuration into as json file

In [15]:
file_json = Path(temp_folder.name) / "edb_configuration.json"
with open(file_json, "w") as f:
    json.dump(cfg, f, indent=4, ensure_ascii=False)

Equivalent toml file looks like below 

In [16]:
toml_string = toml.dumps(cfg)
print(toml_string)

[[pin_groups]]
name = "U9_5V_1"
reference_designator = "U9"
pins = [ "32", "33",]

[[pin_groups]]
name = "U9_GND"
reference_designator = "U9"
net = "GND"

[[ports]]
name = "port_1"
reference_designator = "X1"
type = "circuit"

[ports.positive_terminal]
net = "PCIe_Gen4_TX2_N"
[ports.negative_terminal]
net = "GND"
[[ports]]
name = "port_2"
reference_designator = "C375"
type = "circuit"

[ports.positive_terminal]
pin = "1"
[ports.negative_terminal]
pin = "2"
[[ports]]
name = "port_3"
type = "circuit"

[ports.positive_terminal]
pin_group = "U9_5V_1"
[ports.negative_terminal]
pin_group = "U9_GND"
[[ports]]
name = "port_4"
type = "circuit"

[ports.positive_terminal.coordinates]
layer = "1_Top"
point = [ "104mm", "37mm",]
net = "AVCC_1V3"
[ports.negative_terminal.coordinates]
layer = "Inner6(GND2)"
point = [ "104mm", "37mm",]
net = "GND"
[[ports]]
name = "port_5"
reference_designator = "U1"
type = "coax"

[ports.positive_terminal]
pin = "AM17"
[[ports]]
name = "port_6"
reference_designator =

## Import configuration into example layout

In [17]:
edbapp.configuration.load(config_file=file_json)
edbapp.configuration.run()

PyEDB INFO: Updating nets finished. Time lapse 0:00:00


PyEDB INFO: Updating components finished. Time lapse 0:00:00


PyEDB INFO: Creating pin groups finished. Time lapse 0:00:00.016144


PyEDB INFO: Placing sources finished. Time lapse 0:00:00


PyEDB INFO: Applying materials finished. Time lapse 0:00:00


...

PyEDB INFO: Placing ports finished. Time lapse 0:00:00.489357


PyEDB INFO: Placing terminals completed in 0.0000 seconds.


PyEDB INFO: Placing probes finished. Time lapse 0:00:00


PyEDB INFO: Applying operations completed in 0.0000 seconds.


True

## Review

In [18]:
edbapp.ports

{'port_1': <pyedb.dotnet.database.edb_data.ports.CircuitPort at 0x24517ade560>,
 'port_2': <pyedb.dotnet.database.edb_data.ports.CircuitPort at 0x24517ade200>,
 'port_3': <pyedb.dotnet.database.edb_data.ports.CircuitPort at 0x24517ade4d0>,
 'port_4': <pyedb.dotnet.database.edb_data.ports.CircuitPort at 0x24517adffd0>,
 'port_5': <pyedb.dotnet.database.edb_data.ports.CoaxPort at 0x24517adf8e0>,
 'port_6': <pyedb.dotnet.database.edb_data.ports.CircuitPort at 0x24517adec20>,
 'U7_VDD_DDR_T9': <pyedb.dotnet.database.edb_data.ports.CircuitPort at 0x24517aded10>,
 'U7_VDD_DDR_R1': <pyedb.dotnet.database.edb_data.ports.CircuitPort at 0x24517ade8c0>,
 'U7_VDD_DDR_L9': <pyedb.dotnet.database.edb_data.ports.CircuitPort at 0x24517ade1d0>,
 'U7_VDD_DDR_L1': <pyedb.dotnet.database.edb_data.ports.CircuitPort at 0x24517ade2f0>,
 'U7_VDD_DDR_J9': <pyedb.dotnet.database.edb_data.ports.CircuitPort at 0x24517ade2c0>,
 'U7_VDD_DDR_J8': <pyedb.dotnet.database.edb_data.ports.CircuitPort at 0x24517ade260>,
 

## Save and close Edb
The temporary folder will be deleted once the execution of this script is finished. Replace **edbapp.save()** with
**edbapp.save_as("C:/example.aedb")** to keep the example project.

In [19]:
edbapp.save()
edbapp.close()

PyEDB INFO: Save Edb file completed in 0.1267 seconds.


PyEDB INFO: Close Edb file completed in 0.0636 seconds.


True